In [1]:
import os

import pandas as pd
from scipy.misc import imread, imsave, imresize
from tqdm import tqdm
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/illarionkhliestov/PycharmEnvs/pytorch/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['imsave', 'imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [58]:
def gray_to_color(image):
    return np.repeat(image[:, :, np.newaxis], 3, axis=2)

def resize_and_save(in_folder, out_folder, train=True, category_idx=None):
    df_list = []
    for image_fname in os.listdir(in_folder):
        if not image_fname.endswith('.jpg'):
            continue
        if image_fname.startswith('.'):
            continue
        image_path = os.path.join(in_folder, image_fname)
        if os.path.isdir(image_path):
            continue
        image = imread(image_path)
        if len(image.shape) != 3:
            image = gray_to_color(image)
        # resize image
        image = imresize(image, (64, 64), interp='bilinear')
        # save new image
        new_image_path = os.path.join(out_folder, image_fname)
        imsave(new_image_path, image)
        if train:
            df_list.append({
                'path': new_image_path,
                'class_': category_idx
            })
        else:
            df_list.append({'path': new_image_path})
    return df_list
    
train_folder = '/Users/illarionkhliestov/Datasets/caltech-256/train/'
train_out_folder = '/Users/illarionkhliestov/Datasets/caltech-256/train_resized/'
df_list = []
for category_dir in tqdm(os.listdir(train_folder)):
    if category_dir.startswith('.'):
        continue
    category_idx, category_name = category_dir.split('.')
    category_idx = int(category_idx)
    category_path = os.path.join(train_folder, category_dir)
    category_out_folder = os.path.join(train_out_folder, str(category_idx))
    os.makedirs(category_out_folder, exist_ok=True)
    df_list.extend(resize_and_save(category_path, category_out_folder, train=True, category_idx=category_idx))
        
df = pd.DataFrame.from_dict(df_list)
df.to_csv('/Users/illarionkhliestov/Datasets/caltech-256/train.csv')

test_folder = '/Users/illarionkhliestov/Datasets/caltech-256/test/'
test_out_folder = '/Users/illarionkhliestov/Datasets/caltech-256/test_resized/'
os.makedirs(test_out_folder, exist_ok=True)
df_list = resize_and_save(test_folder, test_out_folder)
df = pd.DataFrame.from_dict(df_list)
df.to_csv('/Users/illarionkhliestov/Datasets/caltech-256/test.csv')

100%|██████████| 258/258 [04:13<00:00,  3.01s/it]


In [2]:
# split on train and test datasets
df = pd.read_csv('/Users/illarionkhliestov/Datasets/caltech-256/train.csv', index_col=0)

In [28]:
val_df = df.groupby('class_').head(20)
val_df.to_csv('/Users/illarionkhliestov/Datasets/caltech-256/df_val_split.csv')
val_df_pathes = val_df.path.unique().tolist()
train_df = df[~df.path.isin(val_df_pathes)]
train_df.to_csv('/Users/illarionkhliestov/Datasets/caltech-256/df_train_split.csv')

In [13]:
df.columns.values

array(['class_', 'path'], dtype=object)

In [24]:
df['class_'].min()

1

In [27]:
train_df.shape[0] / 16 / 100

12.704375